## nick comment on run model
The main script we use to run the experiment is  `schema_prediction_task_9_8_20.batch_exp`.  It's a wrapper function for a lot of code,
but it will generate a set of randomly sampled tasks of the specified conditions (default = Blocked, Interleaved), run the model (either SEM or a "no_split" variant of SEM that collapses to the NN specified) and calculate the meaningful metrics of the task.


It's desined to be a parallelizable job on a cluster -- you can pass a set of parameters to the function and get a pair of files as an output that represents a random sample of behavior for those parameters on the specified tasks.  I (NTF) would spawn thousands of instances of this function in seperate SLURM jobs, each with different parameters and write the results to file for grid-searches.  It is possible to get fancier with the paralization by writting a custom wrapper for some of the SEM module (look at the function `no_split_sem.no_split_sem_run_with_boundaries` for ideas) but I chose not to.

This script runs one param_set
outputs results and trialxtrial

## Load Libraries

In [1]:
import sys
print("Python version")
print (sys.version)

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from tqdm import tqdm
from sklearn import metrics
import pandas as pd

from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from scipy.special import logsumexp
from scipy.stats import norm
from glob import glob

Python version
3.7.9 (default, Aug 31 2020, 07:22:35) 
[Clang 10.0.0 ]


In [2]:
from schema_prediction_task_9_8_20 import generate_exp, batch_exp
from vanilla_lstm import VanillaLSTM
from sem.event_models import NonLinearEvent

TensorFlow Version: 2.3.1


## Story Parameters

In [3]:
seed = None
err = 0.2; # error probability for plate's formula

# story_generator = generate_exp_balanced
""" 
actor_weight & instructions_weight 
control instruction condition
"""
story_kwargs = dict(seed=None, err=0.2, actor_weight=1.0, instructions_weight=1.0)

x, y, e, embedding_library = generate_exp('blocked', **story_kwargs)
print(np.shape(x)[-1])

10


### gridsearch params 

In [4]:
lr                = 0.05
n_batch=1
no_split = False # toggle between SEM (False) and LSTM (True)

## SEM Parameters

In [5]:
## sem parameters
dropout           = 0.0  # don't change
l2_regularization = 0.0  # don't change
n_epochs          = 2
batch_size        = 25   # don't change

epsilon           = 1e-5 # don't change
log_alpha         = 0.0  # sCRP alpha is set in log scale
log_lambda        = 0.0   # sCRP lambda is set in log scale
n_hidden          = None # don't change -- set the number of hidden units equal to the dimensionality
batch_update      = False # batch update (True) or local gradient updating (False)


# ADAM parameters, don't change
optimizer_kwargs = dict(
    lr=lr, beta_1=0.9, beta_2=0.999, epsilon=epsilon, amsgrad=False
)

# set the event model class
f_class = VanillaLSTM

# the dictionaries below are used to pass the above parameters to SEM
f_opts=dict(
  batch_size=batch_size, 
  batch_update=batch_update, 
  dropout=dropout,
  l2_regularization=l2_regularization, 
  n_epochs=n_epochs,
    optimizer_kwargs=optimizer_kwargs
)

sem_kwargs = dict(lmda=np.exp(log_lambda), alfa=np.exp(log_alpha), f_opts=f_opts, f_class=f_class)


# Run model

main fun call

In [ ]:
""" 
batch_exp main fun call
"""
results, trialXtrial, _ = batch_exp(
              sem_kwargs, story_kwargs, 
              no_split=no_split, block_only=False, run_instructed=False, 
              sem_progress_bar=True, progress_bar=False,
              n_batch=n_batch, 
)


# convert from JSON file format (dict) to pandas df
results = pd.DataFrame(results)
trialXtrial = pd.DataFrame(trialXtrial)

Run SEM:  56%|█████▋    | 113/200 [01:14<01:16,  1.14it/s]

# save


In [ ]:
save_fpath = None
results,trialXtrial